In [442]:
import numpy as np

class LinearCode:

    def __init__(self, matrix):
        self.matrix = matrix

    # Приведение матрицы к ступенчатому виду
    def REF(self):

        rows = len(self.matrix)
        cols = len(self.matrix[0])
        current_row = 0

        if rows < cols:
            zeros = np.zeros((cols - rows, cols), dtype=int)
            self.matrix = np.vstack((self.matrix, zeros))
                

        for col in range(cols):
            leading_row = current_row
            while leading_row < rows and self.matrix[leading_row][col] == 0:
                leading_row += 1

            if leading_row == rows:
                continue
            self.matrix[[current_row, leading_row], :] = self.matrix[[leading_row, current_row], :]

            for row in range(current_row + 1, rows):
                if self.matrix[row][col] == 1:
                    for j in range(cols):
                        self.matrix[row][j] ^= self.matrix[current_row][j]

            current_row += 1
        
        non_zero_rows = [row for row in self.matrix if not np.all(row == 0)]
        self.matrix = np.array(non_zero_rows)
        
        return self.matrix
    
    # Приведение матрицы к приведённому ступенчатому виду
    def RREF(self):
        rows = len(self.matrix)
        cols = len(self.matrix[0])

        if rows < cols:
            zeros = np.zeros((cols - rows, cols), dtype=int)
            self.matrix = np.vstack((self.matrix, zeros))

        current_row = 0
        for col in range(cols):
            leading_row = current_row
            while leading_row < rows and self.matrix[leading_row, col] == 0:
                leading_row += 1

            if leading_row == rows:
                continue
            self.matrix[[current_row, leading_row], :] = self.matrix[[leading_row, current_row], :]

            for row in range(current_row - 1, -1, -1):
                if self.matrix[row, col] == 1:
                    self.matrix[row, :] ^= self.matrix[current_row, :]

            for row in range(current_row + 1, rows):
                if self.matrix[row, col] == 1:
                    self.matrix[row, :] ^= self.matrix[current_row, :]

            current_row += 1

        non_zero_rows = [row for row in self.matrix if not np.all(row == 0)]
        self.matrix = np.array(non_zero_rows)

        return self.matrix

    # Поиск порождающей матрицы
    def find_generator_matrix(self):
        G = np.array(self.matrix)  
        G = self.REF() 
        return G
    
    # Получение числа строк и столбцов матрицы 
    def get_num_of_rows_and_columns(self):
        G = np.array(self.matrix)
        G = G[~np.all(G == 0, axis=1)] 
        return f"n={G.shape[1]}", f"k={G.shape[0]}"

    # Формирование проверочной матрицы
    def form_verification_matrix(self):
        temporary_RREF_matrix = self.RREF()
        lead_column_indexes = []
        for row in temporary_RREF_matrix:
            for i, element in enumerate(row):
                if element == 1:
                    lead_column_indexes.append(i)
                    break
        shorted_matrix = np.delete(temporary_RREF_matrix, lead_column_indexes, axis=1)
        
        size = shorted_matrix.shape[1]
        identity_matrix = np.identity(size)

        verification_matrix = np.zeros((size+shorted_matrix.shape[0], size)) 

        for i, col in enumerate(lead_column_indexes):
            verification_matrix[col] = shorted_matrix[i]

        temp_index = 0
        for i in range(size+shorted_matrix.shape[0]):
            if i not in lead_column_indexes:
                verification_matrix[i] = identity_matrix[temp_index]
                temp_index += 1

        return verification_matrix
        
        
    # Генерация кодовых слов путём сложения слов из порождающего множества
    def generate_codewords_method1(self):
        row, col = self.matrix.shape
        result = []
        for i in range(1, 2 ** row):
            bin = [int(x) for x in format(i, f'0{row}b')]
            word = np.dot(bin, self.matrix) % 2
            result.append(word)
        unique_words = np.unique(result, axis=0)
        return unique_words
    
    # Генерация кодовых слов путём умножения двоичных слов на порождающую матрицу
    def generate_codewords_method2(self):
        row, col = self.matrix.shape
        result = []
        for i in range(2 ** row):
            bin = np.array([int(x) for x in format(i, f'0{row}b')])
            word = np.dot(bin, self.matrix) % 2
            result.append(word)
        return np.array(result)
    
    # Расстояние Хемминга
    def hemming_distance(self):
        code_words = self.generate_codewords_method1()
        weight = min([np.sum(word) for word in code_words if np.sum(word) > 0])
        return weight
    
    # Обнаружение ошибки
    def find_errors(self, verification_matrix, t):
        code_words = self.generate_codewords_method1()
        word = code_words[0]
        error = np.zeros(word.shape, dtype=int)
        error[:t] = 1
        error_word = (word + error) % 2
        return word, error, error_word, np.dot(error_word, verification_matrix) % 2

In [443]:
mat = [
    [1, 1, 1, 0, 1],
    [1, 0, 1, 1, 0],
]
o = LinearCode(mat)

ref = o.REF()

print('Ступенчатая матрица: \n')
for row in ref:
    print(row)

rref = o.RREF()

print ('\nПриведённая ступенчатая матрица:\n')
for row in rref:
    print(row)


Ступенчатая матрица: 

[1 1 1 0 1]
[0 1 0 1 1]

Приведённая ступенчатая матрица:

[1 0 1 1 0]
[0 1 0 1 1]


In [444]:
s = np.array([
[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
[0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
[1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
[0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
])

o = LinearCode(s)
G = o.find_generator_matrix()
print("Порождающая матрица: ")
print(G)
print("\nЧисло строк и столбцов без учёта нулевых:")
print(o.get_num_of_rows_and_columns())

Порождающая матрица: 
[[1 0 1 1 0 0 0 1 0 0 1]
 [0 0 0 1 1 1 0 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1]]

Число строк и столбцов без учёта нулевых:
('n=11', 'k=5')


In [445]:
matrix = [
[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
[0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
]
l = LinearCode(matrix)
l.form_verification_matrix()

array([[0., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [446]:
G = np.array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
[0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

mat = LinearCode(G)
print('Метод 1')
a = mat.generate_codewords_method1()
a

Метод 1


array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
       [1, 0, 1, 1, 0, 0, 0, 1, 0,

In [447]:
b = mat.form_verification_matrix()
b

array([[0., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [448]:
print('Метод 2')
second = mat.generate_codewords_method2()
second

Метод 2


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 1, 0, 1,

In [449]:
# Проверка 1-го метода
results = []
for word in a:
  result = np.dot(word, b) % 2
  results.append(result)
  print(f"Результаты умножения кодовых слов на H:\n{results}")
  if not np.all(result == 0):
     print("Ошибка! Некоторые кодовые слова не умножаются на H в нулевой вектор.")
  else: 
     print("Всё верно, нулевой вектор")
  print('Итоговый результат\n', results)


Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.])]
Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0.

In [450]:
# Проверка 2-го метода
results = []
for word in second:
  result = np.dot(word, b) % 2
  results.append(result)
  print(f"Результаты умножения кодовых слов на H:\n{results}")
  if not np.all(result == 0):
     print("Ошибка! Некоторые кодовые слова не умножаются на H в нулевой вектор.")
  else: 
     print("Всё верно, нулевой вектор")
  print('Итоговый результат\n', results)

Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.])]
Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Результаты умножения кодовых слов на H:
[array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.])]
Всё верно, нулевой вектор
Итоговый результат
 [array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0.

In [451]:
print('Расстояние Хэмминга: ', mat.hemming_distance())
print('Кратность обнаруживаемой ошибки: ', mat.hemming_distance() - 1)

Расстояние Хэмминга:  2
Кратность обнаруживаемой ошибки:  1


In [452]:
error1 = mat.find_errors(b, 1)
error1

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], dtype=int32),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], dtype=int32),
 array([0., 1., 1., 1., 1., 0.]))